Paper: Siamese Neural Networks for One-shot Image Recognition
http://www.cs.cmu.edu/~rsalakhu/papers/oneshot1.pdf

Images sizes: 40x40 / 75x75 / 105x105 / 120x120 / 150x150
Images in RGB / Grayscale

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
import math
import wandb
from wandb.keras import WandbCallback
from helper_functions import plot_training
from helper_functions import create_tf_data_datasets_contrastive
from helper_functions import create_tf_data_testset_contrastive
from helper_functions import get_classification_report

# Original Model

## First Run - 30k Pairs

In [204]:
anchor_images_path = "npz_datasets/pairs_30k_224_224/anchor"
positive_images_path = "npz_datasets/pairs_30k_224_224/positive"
width, height, channels = 105, 105, 3
batch_size = 128
train_dataset, val_dataset = create_tf_data_datasets_contrastive(anchor_images_path, positive_images_path, batch_size, height, width, rgb=True)

In [3]:
def get_original_model(height, width, channels):

    input = keras.layers.Input(shape=(height, width, channels))

    x = keras.layers.Conv2D(64, (10,10), activation="relu",
                            kernel_initializer=keras.initializers.RandomNormal(mean=0, stddev=0.01),
                            bias_initializer=keras.initializers.RandomNormal(mean=0, stddev=0.01),
                            kernel_regularizer=keras.regularizers.l2(2e-4), name='Conv1')(input)
    x = keras.layers.MaxPooling2D(2,2)(x)

    x = keras.layers.Conv2D(128, (7,7), activation="relu",
                            kernel_initializer=keras.initializers.RandomNormal(mean=0, stddev=0.01),
                            bias_initializer=keras.initializers.RandomNormal(mean=0, stddev=0.01),
                            kernel_regularizer=keras.regularizers.l2(2e-4), name='Conv2')(x)
    x = keras.layers.MaxPooling2D(2,2)(x)

    x = keras.layers.Conv2D(128, (4,4), activation="relu",
                        kernel_initializer=keras.initializers.RandomNormal(mean=0, stddev=0.01),
                        bias_initializer=keras.initializers.RandomNormal(mean=0, stddev=0.01),
                        kernel_regularizer=keras.regularizers.l2(2e-4), name='Conv3')(x)
    x = keras.layers.MaxPooling2D(2,2)(x)

    x = keras.layers.Conv2D(256, (4,4), activation="relu",
                        kernel_initializer=keras.initializers.RandomNormal(mean=0, stddev=0.01),
                        bias_initializer=keras.initializers.RandomNormal(mean=0, stddev=0.01),
                        kernel_regularizer=keras.regularizers.l2(2e-4), name='Conv4')(x)

    x = keras.layers.Flatten()(x)
    output = keras.layers.Dense(4096, activation="sigmoid", kernel_initializer=keras.initializers.RandomNormal(mean=0, stddev=0.2),
                           bias_initializer=keras.initializers.RandomNormal(mean=0, stddev=0.01),
                           kernel_regularizer=keras.regularizers.l2(1e-3), name='Dense1')(x)

    model = keras.models.Model(input, output)

    return model

In [25]:
original_model = get_original_model(height,width,channels)

In [26]:
original_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 105, 105, 3)]     0         
_________________________________________________________________
Conv1 (Conv2D)               (None, 96, 96, 64)        19264     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 48, 48, 64)        0         
_________________________________________________________________
Conv2 (Conv2D)               (None, 42, 42, 128)       401536    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 21, 21, 128)       0         
_________________________________________________________________
Conv3 (Conv2D)               (None, 18, 18, 128)       262272    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 9, 9, 128)         0   

In [27]:
left_input = keras.layers.Input(shape=(height, width, channels))
right_input = keras.layers.Input(shape=(height, width, channels))

encoded_l = original_model(left_input)
encoded_r = original_model(right_input)

L1_layer = keras.layers.Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))
merge_layer = L1_layer([encoded_l, encoded_r])

prediction = keras.layers.Dense(1, activation="sigmoid")(merge_layer)

original_siamese_model = keras.models.Model([left_input, right_input], outputs=prediction)

In [ ]:
run = wandb.init(project="Architecture_1",
                 config={"learning_rate": 0.001,
                         "momentum": 0.5,
                         "otimizer": "SGD",
                         "loss_function": "binary_crossentropy",
                         "epochs": 200,
                         "architecture": "Original - 30k"})

In [29]:
config = wandb.config

In [30]:
original_siamese_model.compile(loss=config.loss_function,
                               optimizer=keras.optimizers.SGD(learning_rate=config.learning_rate, momentum=config.momentum), metrics=["accuracy"])

In [32]:
def scheduler(epoch, lr):
    new_lr =  lr * 0.99
    print(f"learning rate scheduled to {new_lr}")
    return new_lr

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="Architecture_1_Checkpoints/Original_30k",
    save_weights_only=True,
    monitor="val_accuracy",
    save_best_only=True
)

In [33]:
history_original_siamese_model = original_siamese_model.fit(train_dataset, epochs=config.epochs, validation_data=val_dataset, callbacks=[tf.keras.callbacks.LearningRateScheduler(scheduler),
                                                                                                                              model_checkpoint_callback, WandbCallback()])

Epoch 1/200
learning rate scheduled to 0.0009900000470224768
166/166 [==============================] - 14s 73ms/step - loss: 1509.8461 - accuracy: 0.5195 - val_loss: 1508.8473 - val_accuracy: 0.5477
Epoch 2/200
learning rate scheduled to 0.000980100086890161
166/166 [==============================] - 13s 72ms/step - loss: 1507.8730 - accuracy: 0.5517 - val_loss: 1506.8850 - val_accuracy: 0.5713
Epoch 3/200
learning rate scheduled to 0.0009702991275116801
166/166 [==============================] - 13s 72ms/step - loss: 1505.9220 - accuracy: 0.5641 - val_loss: 1504.9448 - val_accuracy: 0.5760
Epoch 4/200
learning rate scheduled to 0.0009605961316265165
166/166 [==============================] - 13s 72ms/step - loss: 1503.9926 - accuracy: 0.5570 - val_loss: 1503.0267 - val_accuracy: 0.5584
Epoch 5/200
learning rate scheduled to 0.0009509901772253215
166/166 [==============================] - 13s 72ms/step - loss: 1502.0851 - accuracy: 0.5658 - val_loss: 1501.1294 - val_accuracy: 0.5626
E

In [34]:
loss, accuracy = original_siamese_model.evaluate(val_dataset)
wandb.log({'Test Error Rate': round((1-accuracy)*100, 2)})

42/42 [==============================] - 2s 23ms/step - loss: 1348.0594 - accuracy: 0.7954


In [206]:
anchor_images_path = "npz_datasets/test_pairs_224_224/anchor"
positive_images_path = "npz_datasets/test_pairs_224_224/positive"
test_dataset = create_tf_data_testset_contrastive(anchor_images_path, positive_images_path, height, width, rgb=True)

In [195]:
precision, recall, f1_score, preds_wandb, labels = get_classification_report(test_dataset, original_siamese_model)

In [190]:
wandb.log({"roc": wandb.plot.roc_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({"pr": wandb.plot.pr_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({'Precision': precision})
wandb.log({'Recall': recall})
wandb.log({'F1 - Score': f1_score})

In [ ]:
run.finish()

## Second Run - 90k Pairs

In [207]:
anchor_images_path = "npz_datasets/pairs_90k_224_224/anchor"
positive_images_path = "npz_datasets/pairs_90k_224_224/positive"
width, height, channels = 105, 105, 3
batch_size = 128
train_dataset, val_dataset = create_tf_data_datasets_contrastive(anchor_images_path, positive_images_path, batch_size, height, width, rgb=True)

In [208]:
original_model = get_original_model(height,width,channels)
original_model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 105, 105, 3)]     0         
_________________________________________________________________
Conv1 (Conv2D)               (None, 96, 96, 64)        19264     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 48, 48, 64)        0         
_________________________________________________________________
Conv2 (Conv2D)               (None, 42, 42, 128)       401536    
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 21, 21, 128)       0         
_________________________________________________________________
Conv3 (Conv2D)               (None, 18, 18, 128)       262272    
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 9, 9, 128)         0   

In [210]:
left_input = keras.layers.Input(shape=(height, width, channels))
right_input = keras.layers.Input(shape=(height, width, channels))

encoded_l = original_model(left_input)
encoded_r = original_model(right_input)

L1_layer = keras.layers.Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))
merge_layer = L1_layer([encoded_l, encoded_r])

prediction = keras.layers.Dense(1, activation="sigmoid")(merge_layer)

original_siamese_model_2 = keras.models.Model([left_input, right_input], outputs=prediction)

In [211]:
run = wandb.init(project="Architecture_1",
                 config={"learning_rate": 0.001,
                         "momentum": 0.5,
                         "otimizer": "SGD",
                         "loss_function": "binary_crossentropy",
                         "epochs": 200,
                         "architecture": "Original - 90k"})

In [212]:
config = wandb.config

In [213]:
original_siamese_model_2.compile(loss=config.loss_function,
                               optimizer=keras.optimizers.SGD(learning_rate=config.learning_rate, momentum=config.momentum), metrics=["accuracy"])

In [214]:
def scheduler(epoch, lr):
    new_lr =  lr * 0.99
    print(f"learning rate scheduled to {new_lr}")
    return new_lr

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="Architecture_1_Checkpoints/Original_90k",
    save_weights_only=True,
    monitor="val_accuracy",
    save_best_only=True
)

In [215]:
history_original_siamese_model_2 = original_siamese_model_2.fit(train_dataset, epochs=config.epochs, validation_data=val_dataset, callbacks=[tf.keras.callbacks.LearningRateScheduler(scheduler),
                                                                                                                                         model_checkpoint_callback, WandbCallback()])

Epoch 1/200
learning rate scheduled to 0.0009900000470224768
610/610 [==============================] - 45s 64ms/step - loss: 1507.0350 - accuracy: 0.5532 - val_loss: 1503.3882 - val_accuracy: 0.5779
Epoch 2/200
learning rate scheduled to 0.000980100086890161
610/610 [==============================] - 42s 63ms/step - loss: 1499.8073 - accuracy: 0.5696 - val_loss: 1496.2098 - val_accuracy: 0.5619
Epoch 3/200
learning rate scheduled to 0.0009702991275116801
610/610 [==============================] - 42s 62ms/step - loss: 1492.6814 - accuracy: 0.5683 - val_loss: 1489.1345 - val_accuracy: 0.5802
Epoch 4/200
learning rate scheduled to 0.0009605961316265165
610/610 [==============================] - 42s 63ms/step - loss: 1485.6611 - accuracy: 0.5803 - val_loss: 1482.1699 - val_accuracy: 0.5906
Epoch 5/200
learning rate scheduled to 0.0009509901772253215
610/610 [==============================] - 42s 63ms/step - loss: 1478.7444 - accuracy: 0.5917 - val_loss: 1475.3059 - val_accuracy: 0.5928
E

In [216]:
loss, accuracy = original_siamese_model_2.evaluate(val_dataset)
wandb.log({'Test Error Rate': round((1-accuracy)*100, 2)})

153/153 [==============================] - 7s 20ms/step - loss: 994.0361 - accuracy: 0.8654


In [217]:
anchor_images_path = "npz_datasets/test_pairs_224_224/anchor"
positive_images_path = "npz_datasets/test_pairs_224_224/positive"
test_dataset = create_tf_data_testset_contrastive(anchor_images_path, positive_images_path, height, width, rgb=True)

In [218]:
precision, recall, f1_score, preds_wandb, labels = get_classification_report(test_dataset, original_siamese_model_2)

In [219]:
wandb.log({"roc": wandb.plot.roc_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({"pr": wandb.plot.pr_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({'Precision': precision})
wandb.log({'Recall': recall})
wandb.log({'F1 - Score': f1_score})

In [ ]:
run.finish()

## Third Run - 150k Pairs

In [4]:
anchor_images_path = "npz_datasets/pairs_150k_224_224/anchor"
positive_images_path = "npz_datasets/pairs_150k_224_224/positive"
width, height, channels = 105, 105, 3
batch_size = 128
train_dataset, val_dataset = create_tf_data_datasets_contrastive(anchor_images_path, positive_images_path, batch_size, height, width, rgb=True)

In [5]:
original_model = get_original_model(height,width,channels)
original_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 105, 105, 3)]     0         
_________________________________________________________________
Conv1 (Conv2D)               (None, 96, 96, 64)        19264     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 48, 48, 64)        0         
_________________________________________________________________
Conv2 (Conv2D)               (None, 42, 42, 128)       401536    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 21, 21, 128)       0         
_________________________________________________________________
Conv3 (Conv2D)               (None, 18, 18, 128)       262272    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 9, 9, 128)         0     

In [6]:
left_input = keras.layers.Input(shape=(height, width, channels))
right_input = keras.layers.Input(shape=(height, width, channels))

encoded_l = original_model(left_input)
encoded_r = original_model(right_input)

L1_layer = keras.layers.Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))
merge_layer = L1_layer([encoded_l, encoded_r])

prediction = keras.layers.Dense(1, activation="sigmoid")(merge_layer)

original_siamese_model_3 = keras.models.Model([left_input, right_input], outputs=prediction)

In [7]:
run = wandb.init(project="Architecture_1",
                 config={"learning_rate": 0.001,
                         "momentum": 0.5,
                         "otimizer": "SGD",
                         "loss_function": "binary_crossentropy",
                         "epochs": 200,
                         "architecture": "Original - 150k"})

wandb: Currently logged in as: schauppi (use `wandb login --relogin` to force relogin)


In [8]:
config = wandb.config

In [9]:
original_siamese_model_3.compile(loss=config.loss_function,
                                 optimizer=keras.optimizers.SGD(learning_rate=config.learning_rate, momentum=config.momentum), metrics=["accuracy"])

In [10]:
def scheduler(epoch, lr):
    new_lr =  lr * 0.99
    print(f"learning rate scheduled to {new_lr}")
    return new_lr

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="Architecture_1_Checkpoints/Original_150k",
    save_weights_only=True,
    monitor="val_accuracy",
    save_best_only=True
)

In [11]:
history_original_siamese_model_3 = original_siamese_model_3.fit(train_dataset, epochs=config.epochs, validation_data=val_dataset, callbacks=[tf.keras.callbacks.LearningRateScheduler(scheduler),
                                                                                                                                             model_checkpoint_callback, WandbCallback()])

Epoch 1/200
learning rate scheduled to 0.0009900000470224768
1014/1014 [==============================] - 78s 67ms/step - loss: 1504.4772 - accuracy: 0.5708 - val_loss: 1498.4337 - val_accuracy: 0.5637
Epoch 2/200
learning rate scheduled to 0.000980100086890161
1014/1014 [==============================] - 71s 64ms/step - loss: 1492.4955 - accuracy: 0.5783 - val_loss: 1486.5610 - val_accuracy: 0.5823
Epoch 3/200
learning rate scheduled to 0.0009702991275116801
1014/1014 [==============================] - 72s 65ms/step - loss: 1480.7300 - accuracy: 0.5948 - val_loss: 1474.8993 - val_accuracy: 0.6120
Epoch 4/200
learning rate scheduled to 0.0009605961316265165
1014/1014 [==============================] - 73s 66ms/step - loss: 1469.1669 - accuracy: 0.6262 - val_loss: 1463.4354 - val_accuracy: 0.6393
Epoch 5/200
learning rate scheduled to 0.0009509901772253215
1014/1014 [==============================] - 72s 65ms/step - loss: 1457.7999 - accuracy: 0.6574 - val_loss: 1452.1571 - val_accuracy

In [14]:
loss, accuracy = original_siamese_model_3.evaluate(val_dataset)
wandb.log({'Test Error Rate': round((1-accuracy)*100, 2)})

254/254 [==============================] - 11s 21ms/step - loss: 753.4119 - accuracy: 0.8999


In [12]:
anchor_images_path = "npz_datasets/test_pairs_224_224/anchor"
positive_images_path = "npz_datasets/test_pairs_224_224/positive"
test_dataset = create_tf_data_testset_contrastive(anchor_images_path, positive_images_path, height, width, rgb=True)

In [13]:
precision, recall, f1_score, preds_wandb, labels = get_classification_report(test_dataset, original_siamese_model_3)

In [16]:
wandb.log({"roc": wandb.plot.roc_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({"pr": wandb.plot.pr_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({'Precision': precision})
wandb.log({'Recall': recall})
wandb.log({'F1 - Score': f1_score})

In [ ]:
run.finish()

## Fourth Run - 30k Pairs gray

In [20]:
anchor_images_path = "npz_datasets/pairs_30k_224_224/anchor"
positive_images_path = "npz_datasets/pairs_30k_224_224/positive"
width, height, channels = 105, 105, 1
batch_size = 128
train_dataset, val_dataset = create_tf_data_datasets_contrastive(anchor_images_path, positive_images_path, batch_size, height, width, rgb=False)

In [21]:
original_model = get_original_model(height,width,channels)
original_model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 105, 105, 1)]     0         
_________________________________________________________________
Conv1 (Conv2D)               (None, 96, 96, 64)        6464      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 48, 48, 64)        0         
_________________________________________________________________
Conv2 (Conv2D)               (None, 42, 42, 128)       401536    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 21, 21, 128)       0         
_________________________________________________________________
Conv3 (Conv2D)               (None, 18, 18, 128)       262272    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 9, 9, 128)         0   

In [22]:
left_input = keras.layers.Input(shape=(height, width, channels))
right_input = keras.layers.Input(shape=(height, width, channels))

encoded_l = original_model(left_input)
encoded_r = original_model(right_input)

L1_layer = keras.layers.Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))
merge_layer = L1_layer([encoded_l, encoded_r])

prediction = keras.layers.Dense(1, activation="sigmoid")(merge_layer)

original_siamese_model_4 = keras.models.Model([left_input, right_input], outputs=prediction)

In [12]:
run = wandb.init(project="Architecture_1",
                 config={"learning_rate": 0.001,
                         "momentum": 0.5,
                         "otimizer": "SGD",
                         "loss_function": "binary_crossentropy",
                         "epochs": 200,
                         "architecture": "Original - 30k - Grayscale"})

wandb: Currently logged in as: schauppi (use `wandb login --relogin` to force relogin)


In [13]:
config = wandb.config

In [23]:
original_siamese_model_4.compile(loss=config.loss_function,
                                 optimizer=keras.optimizers.SGD(learning_rate=config.learning_rate, momentum=config.momentum), metrics=["accuracy"])

In [25]:
def scheduler(epoch, lr):
    new_lr =  lr * 0.99
    print(f"learning rate scheduled to {new_lr}")
    return new_lr

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="Architecture_1_Checkpoints/Original_30k_Gray",
    save_weights_only=True,
    monitor="val_accuracy",
    save_best_only=True
)

In [26]:
history_original_siamese_model_4 = original_siamese_model_4.fit(train_dataset, epochs=config.epochs, validation_data=val_dataset, callbacks=[tf.keras.callbacks.LearningRateScheduler(scheduler),
                                                                                                                                             model_checkpoint_callback, WandbCallback()])

Epoch 1/200
learning rate scheduled to 0.0009900000470224768
203/203 [==============================] - 12s 55ms/step - loss: 1509.4403 - accuracy: 0.5015 - val_loss: 1508.2151 - val_accuracy: 0.5015
Epoch 2/200
learning rate scheduled to 0.000980100086890161
203/203 [==============================] - 11s 52ms/step - loss: 1507.0282 - accuracy: 0.5073 - val_loss: 1505.8185 - val_accuracy: 0.5021
Epoch 3/200
learning rate scheduled to 0.0009702991275116801
203/203 [==============================] - 11s 53ms/step - loss: 1504.6437 - accuracy: 0.5033 - val_loss: 1503.4476 - val_accuracy: 0.4926
Epoch 4/200
learning rate scheduled to 0.0009605961316265165
203/203 [==============================] - 11s 52ms/step - loss: 1502.2870 - accuracy: 0.5165 - val_loss: 1501.1044 - val_accuracy: 0.5160
Epoch 5/200
learning rate scheduled to 0.0009509901772253215
203/203 [==============================] - 11s 52ms/step - loss: 1499.9572 - accuracy: 0.5246 - val_loss: 1498.7887 - val_accuracy: 0.5206
E

In [27]:
loss, accuracy = original_siamese_model_4.evaluate(val_dataset)
wandb.log({'Test Error Rate': round((1-accuracy)*100, 2)})

51/51 [==============================] - 2s 18ms/step - loss: 1314.2748 - accuracy: 0.6682


In [28]:
anchor_images_path = "npz_datasets/test_pairs_224_224/anchor"
positive_images_path = "npz_datasets/test_pairs_224_224/positive"
test_dataset = create_tf_data_testset_contrastive(anchor_images_path, positive_images_path, height, width, rgb=False)

In [29]:
precision, recall, f1_score, preds_wandb, labels = get_classification_report(test_dataset, original_siamese_model_4)

In [30]:
wandb.log({"roc": wandb.plot.roc_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({"pr": wandb.plot.pr_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({'Precision': precision})
wandb.log({'Recall': recall})
wandb.log({'F1 - Score': f1_score})

In [ ]:
run.finish()

## Fifth Run - 90K Pairs gray

In [ ]:
anchor_images_path = "npz_datasets/pairs_90k_224_224/anchor"
positive_images_path = "npz_datasets/pairs_90k_224_224/positive"
width, height, channels = 105, 105, 1
batch_size = 128
train_dataset, val_dataset = create_tf_data_datasets_contrastive(anchor_images_path, positive_images_path, batch_size, height, width, rgb=False)

In [ ]:
original_model = get_original_model(height,width,channels)
original_model.summary()

In [ ]:
left_input = keras.layers.Input(shape=(height, width, channels))
right_input = keras.layers.Input(shape=(height, width, channels))

encoded_l = original_model(left_input)
encoded_r = original_model(right_input)

L1_layer = keras.layers.Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))
merge_layer = L1_layer([encoded_l, encoded_r])

prediction = keras.layers.Dense(1, activation="sigmoid")(merge_layer)

original_siamese_model_5 = keras.models.Model([left_input, right_input], outputs=prediction)

In [ ]:
run = wandb.init(project="Architecture_1",
                 config={"learning_rate": 0.001,
                         "momentum": 0.5,
                         "otimizer": "SGD",
                         "loss_function": "binary_crossentropy",
                         "epochs": 200,
                         "architecture": "Original - 90k - Grayscale"})

In [ ]:
config = wandb.config

In [ ]:
original_siamese_model_5.compile(loss=config.loss_function,
                                 optimizer=keras.optimizers.SGD(learning_rate=config.learning_rate, momentum=config.momentum), metrics=["accuracy"])

In [ ]:
def scheduler(epoch, lr):
    new_lr =  lr * 0.99
    print(f"learning rate scheduled to {new_lr}")
    return new_lr

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="Architecture_1_Checkpoints/Original_90k_Gray",
    save_weights_only=True,
    monitor="val_accuracy",
    save_best_only=True
)

In [ ]:
history_original_siamese_model_5 = original_siamese_model_5.fit(train_dataset, epochs=config.epochs, validation_data=val_dataset, callbacks=[tf.keras.callbacks.LearningRateScheduler(scheduler),
                                                                                                                                             model_checkpoint_callback, WandbCallback()])

In [ ]:
loss, accuracy = original_siamese_model_5.evaluate(val_dataset)
wandb.log({'Test Error Rate': round((1-accuracy)*100, 2)})

In [ ]:
anchor_images_path = "npz_datasets/test_pairs_224_224/anchor"
positive_images_path = "npz_datasets/test_pairs_224_224/positive"
test_dataset = create_tf_data_testset_contrastive(anchor_images_path, positive_images_path, height, width, rgb=False)

In [ ]:
precision, recall, f1_score, preds_wandb, labels = get_classification_report(test_dataset, original_siamese_model_5)

In [ ]:
wandb.log({"roc": wandb.plot.roc_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({"pr": wandb.plot.pr_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({'Precision': precision})
wandb.log({'Recall': recall})
wandb.log({'F1 - Score': f1_score})

In [ ]:
run.finish()

## Sixth Run - 150K Pairs Gray

In [18]:
anchor_images_path = "npz_datasets/pairs_150k_224_224/anchor"
positive_images_path = "npz_datasets/pairs_150k_224_224/positive"
width, height, channels = 105, 105, 1
batch_size = 128
train_dataset, val_dataset = create_tf_data_datasets_contrastive(anchor_images_path, positive_images_path, batch_size, height, width, rgb=False)

In [19]:
original_model = get_original_model(height,width,channels)
original_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 105, 105, 1)]     0         
_________________________________________________________________
Conv1 (Conv2D)               (None, 96, 96, 64)        6464      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 48, 48, 64)        0         
_________________________________________________________________
Conv2 (Conv2D)               (None, 42, 42, 128)       401536    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 21, 21, 128)       0         
_________________________________________________________________
Conv3 (Conv2D)               (None, 18, 18, 128)       262272    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 9, 9, 128)         0   

In [20]:
left_input = keras.layers.Input(shape=(height, width, channels))
right_input = keras.layers.Input(shape=(height, width, channels))

encoded_l = original_model(left_input)
encoded_r = original_model(right_input)

L1_layer = keras.layers.Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))
merge_layer = L1_layer([encoded_l, encoded_r])

prediction = keras.layers.Dense(1, activation="sigmoid")(merge_layer)

original_siamese_model_6 = keras.models.Model([left_input, right_input], outputs=prediction)

In [21]:
run = wandb.init(project="Architecture_1",
                 config={"learning_rate": 0.001,
                         "momentum": 0.5,
                         "otimizer": "SGD",
                         "loss_function": "binary_crossentropy",
                         "epochs": 200,
                         "architecture": "Original - 150k - Grayscale"})

In [22]:
config = wandb.config

In [23]:
original_siamese_model_6.compile(loss=config.loss_function,
                                 optimizer=keras.optimizers.SGD(learning_rate=config.learning_rate, momentum=config.momentum), metrics=["accuracy"])

In [24]:
def scheduler(epoch, lr):
    new_lr =  lr * 0.99
    print(f"learning rate scheduled to {new_lr}")
    return new_lr

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="Architecture_1_Checkpoints/Original_150k_Gray",
    save_weights_only=True,
    monitor="val_accuracy",
    save_best_only=True
)

In [25]:
history_original_siamese_model_6 = original_siamese_model_6.fit(train_dataset, epochs=config.epochs, validation_data=val_dataset, callbacks=[tf.keras.callbacks.LearningRateScheduler(scheduler),
                                                                                                                                             model_checkpoint_callback, WandbCallback()])

Epoch 1/200
learning rate scheduled to 0.0009900000470224768
1014/1014 [==============================] - 63s 58ms/step - loss: 1504.8086 - accuracy: 0.5364 - val_loss: 1498.7686 - val_accuracy: 0.5598
Epoch 2/200
learning rate scheduled to 0.000980100086890161
1014/1014 [==============================] - 62s 57ms/step - loss: 1492.8362 - accuracy: 0.5633 - val_loss: 1486.9006 - val_accuracy: 0.5545
Epoch 3/200
learning rate scheduled to 0.0009702991275116801
1014/1014 [==============================] - 58s 54ms/step - loss: 1481.0712 - accuracy: 0.5507 - val_loss: 1475.2389 - val_accuracy: 0.5629
Epoch 4/200
learning rate scheduled to 0.0009605961316265165
1014/1014 [==============================] - 57s 53ms/step - loss: 1469.5118 - accuracy: 0.5697 - val_loss: 1463.7858 - val_accuracy: 0.5808
Epoch 5/200
learning rate scheduled to 0.0009509901772253215
1014/1014 [==============================] - 57s 53ms/step - loss: 1458.1604 - accuracy: 0.5860 - val_loss: 1452.5353 - val_accuracy

In [26]:
loss, accuracy = original_siamese_model_6.evaluate(val_dataset)
wandb.log({'Test Error Rate': round((1-accuracy)*100, 2)})

254/254 [==============================] - 8s 19ms/step - loss: 753.7661 - accuracy: 0.7866


In [27]:
anchor_images_path = "npz_datasets/test_pairs_224_224/anchor"
positive_images_path = "npz_datasets/test_pairs_224_224/positive"
test_dataset = create_tf_data_testset_contrastive(anchor_images_path, positive_images_path, height, width, rgb=False)

In [28]:
precision, recall, f1_score, preds_wandb, labels = get_classification_report(test_dataset, original_siamese_model_6)

In [30]:
wandb.log({"roc": wandb.plot.roc_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({"pr": wandb.plot.pr_curve(labels, preds_wandb, labels=None, classes_to_plot=None)})
wandb.log({'Precision': precision})
wandb.log({'Recall': recall})
wandb.log({'F1 - Score': f1_score})

In [ ]:
run.finish()

# Adapted Model

In [ ]:
def get_adapted_model(height, width, channels):

    input = keras.layers.Input(shape=(height, width, channels))

    x = keras.layers.Conv2D(64, (3,3), activation="relu",
                            kernel_initializer=keras.initializers.RandomNormal(mean=0, stddev=0.01),
                            bias_initializer=keras.initializers.RandomNormal(mean=0, stddev=0.01),
                            kernel_regularizer=keras.regularizers.l2(2e-4), name='Conv1')(input)
    x = keras.layers.Conv2D(64, (3,3), activation="relu",
                            kernel_initializer=keras.initializers.RandomNormal(mean=0, stddev=0.01),
                            bias_initializer=keras.initializers.RandomNormal(mean=0, stddev=0.01),
                            kernel_regularizer=keras.regularizers.l2(2e-4), name='Conv1')(x)
    x = keras.layers.MaxPooling2D(2,2)(x)

    x = keras.layers.Conv2D(128, (3,3), activation="relu",
                            kernel_initializer=keras.initializers.RandomNormal(mean=0, stddev=0.01),
                            bias_initializer=keras.initializers.RandomNormal(mean=0, stddev=0.01),
                            kernel_regularizer=keras.regularizers.l2(2e-4), name='Conv2')(x)
    x = keras.layers.Conv2D(128, (3,3), activation="relu",
                            kernel_initializer=keras.initializers.RandomNormal(mean=0, stddev=0.01),
                            bias_initializer=keras.initializers.RandomNormal(mean=0, stddev=0.01),
                            kernel_regularizer=keras.regularizers.l2(2e-4), name='Conv2')(x)
    x = keras.layers.MaxPooling2D(2,2)(x)

    x = keras.layers.Conv2D(256, (3,3), activation="relu",
                        kernel_initializer=keras.initializers.RandomNormal(mean=0, stddev=0.01),
                        bias_initializer=keras.initializers.RandomNormal(mean=0, stddev=0.01),
                        kernel_regularizer=keras.regularizers.l2(2e-4), name='Conv3')(x)
    x = keras.layers.Conv2D(256, (3,3), activation="relu",
                        kernel_initializer=keras.initializers.RandomNormal(mean=0, stddev=0.01),
                        bias_initializer=keras.initializers.RandomNormal(mean=0, stddev=0.01),
                        kernel_regularizer=keras.regularizers.l2(2e-4), name='Conv3')(x)
    x = keras.layers.Conv2D(256, (3,3), activation="relu",
                        kernel_initializer=keras.initializers.RandomNormal(mean=0, stddev=0.01),
                        bias_initializer=keras.initializers.RandomNormal(mean=0, stddev=0.01),
                        kernel_regularizer=keras.regularizers.l2(2e-4), name='Conv3')(x)
    x = keras.layers.Conv2D(256, (3,3), activation="relu",
                        kernel_initializer=keras.initializers.RandomNormal(mean=0, stddev=0.01),
                        bias_initializer=keras.initializers.RandomNormal(mean=0, stddev=0.01),
                        kernel_regularizer=keras.regularizers.l2(2e-4), name='Conv3')(x)
    x = keras.layers.MaxPooling2D(2,2)(x)

    x = keras.layers.Flatten()(x)
    output = keras.layers.Dense(256, activation="sigmoid", kernel_initializer=keras.initializers.RandomNormal(mean=0, stddev=0.2),
                           bias_initializer=keras.initializers.RandomNormal(mean=0, stddev=0.01),
                           kernel_regularizer=keras.regularizers.l2(1e-3), name='Dense1')(x)

    model = keras.models.Model(input, output)

    return model